##**Find point that minimize the mean of Eculidean distance using torch**

In [1]:
from random import Random
import torch
SEED=5

In [2]:
def gnerate_pnts(N=1):
  random_gen=Random(SEED)
  lst_x,lst_y=[],[]
  for _ in range (N):
    lst_x.append(random_gen.
     uniform(0,1.))
  for _ in range (N):
    lst_y.append(random_gen.uniform(0,1.))
  return lst_x,lst_y

In [3]:
data_x,data_y=gnerate_pnts()
data=torch.tensor([data_x,data_y])
data=data.t()

pnt=torch.tensor([0.3,0.3])
pnt.requires_grad=True
pnt.retain_grad()

for i in range(5):
  print("Iteration: ",i)
  torch_loss=torch.mean(torch.sqrt(((pnt-data)**2).sum(dim=1)))
  torch_loss.backward()
  print(f"torch_grad:{pnt.grad.data}")
  pnt.grad.zero_()
  print(f"torch_grad:{pnt.grad.data}")


Iteration:  0
torch_grad:tensor([-0.5901, -0.8073])
torch_grad:tensor([0., 0.])
Iteration:  1
torch_grad:tensor([-0.5901, -0.8073])
torch_grad:tensor([0., 0.])
Iteration:  2
torch_grad:tensor([-0.5901, -0.8073])
torch_grad:tensor([0., 0.])
Iteration:  3
torch_grad:tensor([-0.5901, -0.8073])
torch_grad:tensor([0., 0.])
Iteration:  4
torch_grad:tensor([-0.5901, -0.8073])
torch_grad:tensor([0., 0.])


##**comp_node class**

In [4]:
class comp_node:
  def __init__(self,val,children=[],op="assign"):
    self.val=val
    self.children=children
    self.op=op
    self.grad=0
    self.backward_prop=lambda: None

  def __to_comp_node(self,obj):
    if not isinstance(obj,comp_node):
      return comp_node(val=obj)
    else:
      return obj
  
  def __sub__(self,other):
    other=self.__to_comp_node(other)
    out=comp_node(val=self.val - other.val,
                  children=[self,other],
                  op="sub")
    def _backward_prop():
      self.grad+= out.grad*(1)
      other.grad+= out.grad*(-1)
    out.backward_prop=_backward_prop
    return out

  def __rsub__(self,other):
    other=self.__to_comp_node(other)
    return other-self

  def __add__(self,other):
    other=self.__to_comp_node(other)
    out=comp_node(val=self.val + other.val,
                  children=[self,other],
                  op="add")    
    def _backward_prop():
      self.grad+= out.grad*(1)
      other.grad+= out.grad*(1)
    out.backward_prop=_backward_prop
    
    return out

  def __radd__(self,other):
    other=self.__to_comp_node(other)
    return other+self

  def __mul__(self,other):
    other=self.__to_comp_node(other)
    out=comp_node(val=self.val * other.val,
                  children=[self,other],
                  op="mult")
    def _backward_prop():
      self.grad+=out.grad*other.val
      other.grad+= out.grad* self.val
    out.backward_prop=_backward_prop

    return out

  def __rmul__(self,other):
    other=self.__to_comp_node(other)
    return other*self

  def __pow__ (self,exponent):
    if not isinstance(exponent,(int,float)):
      raise ValueError ("Unsuported types")
    out=comp_node(val=self.val**exponent,children=[self],op=f"power {exponent}")
    def _backward_prop():
      self.grad+=out.grad*(exponent*self.val**(exponent-1))
    out.backward_prop=_backward_prop
    return out

  def __eq__ (self,other):
    return self.val==other.val

  def __repr__(self):
    return f"val:{self.val:0.4f} | children:{len(self.children)} | op:{self.op} | grad:{self.grad}"

In [5]:
assert comp_node(val=5).val==5,"Assigment failed"

assert (comp_node(val=5)-comp_node(val=3)).val==2
assert (comp_node(val=5)-2.5).val==2.5
assert (5-comp_node(val=3)).val==2

assert (comp_node(val=5)+comp_node(val=3)).val==8
assert (comp_node(val=5)+3).val==8
assert (5+comp_node(val=3)).val==8

assert (comp_node(val=5)*comp_node(val=3)).val==15
assert (comp_node(val=5)*3).val==15
assert (5*comp_node(val=3)).val==15

assert (comp_node(val=5)**2).val==25
assert (comp_node(val=25)**0.5).val==5

assert (comp_node(val=5)== comp_node(val=5))== True

assert (comp_node(val=5)+comp_node(val=3)+comp_node(val=3)).val==11

##**loss_graph**

In [6]:
def loss_graph(x_p,y_p,data_x,data_y):
  I_x=x_p-data_x
  I_y=y_p-data_y
  g_x,g_y=I_x**2,I_y**2
  M=g_x+g_y
  l=M**0.5
  return l ,[l,M,g_x,g_y,I_x,I_y,x_p,y_p]

In [7]:
data_x,data_y=gnerate_pnts(N=1)
x_p,y_p=comp_node(val=0.3),comp_node(val=0.3)
l,rev_topo_order=loss_graph(x_p,y_p,data_x[0],data_y[0])
rev_topo_order[0].grad=1
for (i,node) in enumerate (rev_topo_order):
  node.backward_prop()
  print(i,node)


0 val:0.5472 | children:1 | op:power 0.5 | grad:1
1 val:0.2994 | children:2 | op:add | grad:0.9137222319490423
2 val:0.1043 | children:1 | op:power 2 | grad:0.9137222319490423
3 val:0.1952 | children:1 | op:power 2 | grad:0.9137222319490423
4 val:-0.3229 | children:2 | op:sub | grad:-0.5900849147094943
5 val:-0.4418 | children:2 | op:sub | grad:-0.8073411877467226
6 val:0.3000 | children:0 | op:assign | grad:-0.5900849147094943
7 val:0.3000 | children:0 | op:assign | grad:-0.8073411877467226


## For x_p, y_p, z_p

In [8]:
def gnerate_pnts_3(N=1):
  random_gen=Random(SEED)
  lst_x,lst_y,lst_z=[],[],[]
  for _ in range (N):
    lst_x.append(random_gen.
     uniform(0,1.))
  for _ in range (N):
    lst_y.append(random_gen.uniform(0,1.))
  for _ in range (N):
    lst_z.append(random_gen.uniform(0,1.))    
  return lst_x,lst_y,lst_z

In [9]:
data_x,data_y,data_z=gnerate_pnts_3()
data=torch.tensor([data_x,data_y,data_z])
data=data.t()

pnt=torch.tensor([0.3,0.3,0.3])
pnt.requires_grad=True
pnt.retain_grad()

for i in range(5):
  print("Iteration: ",i)
  torch_loss=torch.mean(torch.sqrt(((pnt-data)**2).sum(dim=1)))
  torch_loss.backward()
  print(f"torch_grad:{pnt.grad.data}")
  pnt.grad.zero_()
  print(f"torch_grad:{pnt.grad.data}")


Iteration:  0
torch_grad:tensor([-0.4375, -0.5986, -0.6710])
torch_grad:tensor([0., 0., 0.])
Iteration:  1
torch_grad:tensor([-0.4375, -0.5986, -0.6710])
torch_grad:tensor([0., 0., 0.])
Iteration:  2
torch_grad:tensor([-0.4375, -0.5986, -0.6710])
torch_grad:tensor([0., 0., 0.])
Iteration:  3
torch_grad:tensor([-0.4375, -0.5986, -0.6710])
torch_grad:tensor([0., 0., 0.])
Iteration:  4
torch_grad:tensor([-0.4375, -0.5986, -0.6710])
torch_grad:tensor([0., 0., 0.])


In [10]:
def loss_graph_3(x_p,y_p,z_p,data_x,data_y,data_z):
  I_x=x_p-data_x
  I_y=y_p-data_y
  I_z = z_p-data_z

  g_x,g_y,g_z=I_x**2,I_y**2,I_z**2
  M0=g_x+g_y
  M=M0+g_z
  l=M**0.5
  return l ,[l,M,M0,g_x,g_y,g_z,I_x,I_y,I_z,x_p,y_p,z_p]

In [11]:
data_x,data_y,data_z=gnerate_pnts_3(N=1)
x_p,y_p,z_p=comp_node(val=0.3),comp_node(val=0.3),comp_node(val=0.3)
l,rev_topo_order=loss_graph_3(x_p,y_p,z_p,data_x[0],data_y[0],data_z[0])
rev_topo_order[0].grad=1
for (i,node) in enumerate (rev_topo_order):
  node.backward_prop()
  print(i,node)


0 val:0.7380 | children:1 | op:power 0.5 | grad:1
1 val:0.5447 | children:2 | op:add | grad:0.6774981199812008
2 val:0.2994 | children:2 | op:add | grad:0.6774981199812008
3 val:0.1043 | children:1 | op:power 2 | grad:0.6774981199812008
4 val:0.1952 | children:1 | op:power 2 | grad:0.6774981199812008
5 val:0.2452 | children:1 | op:power 2 | grad:0.6774981199812008
6 val:-0.3229 | children:2 | op:sub | grad:-0.4375305824530328
7 val:-0.4418 | children:2 | op:sub | grad:-0.5986197093125982
8 val:-0.4952 | children:2 | op:sub | grad:-0.6709854193950939
9 val:0.3000 | children:0 | op:assign | grad:-0.4375305824530328
10 val:0.3000 | children:0 | op:assign | grad:-0.5986197093125982
11 val:0.3000 | children:0 | op:assign | grad:-0.6709854193950939
